In [4]:
import os

print('Process (%s) start' %os.getpid())

pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is (%s).' %(os.getpid(), os.getppid()))
else:
    print('I (%s) just create a child process (%s).' %(os.getpid(),pid))

Process (23409) start
I (23409) just create a child process (24052).
I am child process (24052) and my parent is (23409).


In [12]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.baidu.com'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


In [11]:
import subprocess

print('$ nslookup')
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:', p.returncode)

$ nslookup
Server:		192.168.1.1
Address:	192.168.1.1#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:


Exit code: 0


In [15]:
from multiprocessing import Process,Queue
import os, time, random

def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())
        
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)
        
if __name__=='__main__':
    q = Queue()
    pw = Process(target=write, args=(q,))
    pr = Process(target=read, args=(q,))
    #启动子进程pw，写入
    pw.start()
    #启动子进程pr，读取
    pr.start()
    #等待pw结束
    pw.join()
    #pr进程里是死循环，无法等待其结束，只能强行终止
    pr.terminate()


Process to write: 29739
Put A to queue...
Process to read: 29740
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.


In [20]:
import time, threading

def loop():
    print('thread %s is running...' % threading.current_thread().name)
    n = 0
    while n < 5:
        n = n + 1
        print('thread %s >>> %s' % (threading.current_thread().name, n))
        time.sleep(1)
    print('thread %s ended.' % threading.current_thread().name)

print('thread %s is running...' % threading.current_thread().name)
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


In [16]:
import time, threading
threading.current_thread().name

'MainThread'

In [32]:
import time, threading

balance = 0
lock = threading.Lock()

def change_it(n):
    
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(100000):
        lock.acquire()
        try:
            change_it(n)
        finally:
            lock.release()
    
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


In [17]:
def process_student(name):
    std = Student(name)
    do_task_1(std)
    do_task_2(std)
    
def do_task_1():
    do_subtask_1(std)
    do_subtask_2(std)
    
def do_task_2(std):
    do_subtask_2(std)
    do_subtask_2(std)

In [19]:
import threading

local_school = threading.local()

def process_student():
    std = local_school.student
    print('Hello, %s (in %s)' % (std, threading.current_thread().name))
    
def process_thread(name):
    local_school.student = name
    process_student()
    
    
t1 = threading.Thread(target=process_thread, args=('Alice',),name='Thread-A')

t2 = threading.Thread(target=process_thread, args=('Bob',),name='Thread-B')

t1.start()
t2.start()
t1.join()
t2.join()

Hello, Alice (in Thread-A)Hello, Bob (in Thread-B)



In [2]:
import random, time, queue
from multiprocessing.managers import BaseManager

task_queue = queue.Queue()

result_queue = queue.Queue()

class QueueManager(BaseManager):
    pass

QueueManager.register('get_task_queue',callable=lambda: task_queue)
QueueManager.register('get_result_queue',callable=lambda: result_queue)

manager = QueueManager(address=('',5000), authkey=b'abc')

manager.start()

task = manager.get_task_queue()
result = manager.get_result_queue()

for i in range(10):
    n = random.randint(0, 10000)
    print('Put task %d...' % n)
    task.put(n)
    
print('Try get results...')
for i in range(10):
    r = result.get(timeout=10)
    print('Result: %s' % r)
    
manager.shutdown()
print('master exit.')

Put task 1005...
Put task 5472...
Put task 6954...
Put task 5923...
Put task 7066...
Put task 107...
Put task 6987...
Put task 7336...
Put task 6777...
Put task 2067...
Try get results...
Result: 1005*1005=1010025
Result: 5472*5472=29942784
Result: 6954*6954=48358116
Result: 5923*5923=35081929
Result: 7066*7066=49928356
Result: 107*107=11449
Result: 6987*6987=48818169
Result: 7336*7336=53816896
Result: 6777*6777=45927729
Result: 2067*2067=4272489
master exit.


In [ ]:
import time, sys, queue
from multiprocessing.managers import BaseManager

class QueueManeger(BaseManager):
    pass

QueueManeger.register('get_task_queue')
QueueManeger.register('get_result_queue')

server_addr = '127.0.0.1'
print('Connect to server %s...' % server_addr)

m = QueueManeger(address=(server_addr, 5000), authkey=b'abc')

m.connect()

task = m.get_task_queue()
result = m.get_result_queue()

for i in range(10):
    try:
        n = task.get(timeout=1)
        print('run task %d * %d...' % (n, n))
        r = '%d * %d = %d' % (n, n, n*n)
        time.sleep(1)
        result.put(r)
    except Queue.Empty:
        print('task queue is empty.')
        
print('worker exit.')